In [1]:
import pandas as pd

file_path = "../data_raw/secondary.csv"
datafile = pd.read_csv(file_path)

datafile.head()

,Reference Area,Time Period,Sex,Age group,Units of measurement,Observation Value
0,Afghanistan,2014,All genders,Not applicable,Number,2602734.0
1,Afghanistan,2014,Female,Not applicable,Number,901106.0
2,Afghanistan,2013,Female,Not applicable,Number,874199.0
3,Afghanistan,2013,All genders,Not applicable,Number,2538420.0
4,Afghanistan,2012,All genders,Not applicable,Number,2415884.0


In [2]:
datafile = datafile.rename(columns = {
    "Reference Area" : "Country",
    "Time Period" : "Year",
    "Sex" : "Gender",
    "Observation Value" : "Enrolments"
})

datafile.head()

,Country,Year,Gender,Age group,Units of measurement,Enrolments
0,Afghanistan,2014,All genders,Not applicable,Number,2602734.0
1,Afghanistan,2014,Female,Not applicable,Number,901106.0
2,Afghanistan,2013,Female,Not applicable,Number,874199.0
3,Afghanistan,2013,All genders,Not applicable,Number,2538420.0
4,Afghanistan,2012,All genders,Not applicable,Number,2415884.0


In [3]:
datafile = datafile[datafile["Units of measurement"]=="Number"]

datafile.head(100)

,Country,Year,Gender,Age group,Units of measurement,Enrolments
0,Afghanistan,2014,All genders,Not applicable,Number,2602734.0
1,Afghanistan,2014,Female,Not applicable,Number,901106.0
2,Afghanistan,2013,Female,Not applicable,Number,874199.0
3,Afghanistan,2013,All genders,Not applicable,Number,2538420.0
4,Afghanistan,2012,All genders,Not applicable,Number,2415884.0
...,...,...,...,...,...,...
95,Albania,1993,Female,Not applicable,Number,181277.0
96,Albania,1993,All genders,Not applicable,Number,362439.0
97,Albania,1992,Female,Not applicable,Number,197791.0
98,Albania,1992,All genders,Not applicable,Number,404904.0


In [4]:
datafile = datafile.drop(columns=["Age group", "Units of measurement"])

datafile.head()

,Country,Year,Gender,Enrolments
0,Afghanistan,2014,All genders,2602734.0
1,Afghanistan,2014,Female,901106.0
2,Afghanistan,2013,Female,874199.0
3,Afghanistan,2013,All genders,2538420.0
4,Afghanistan,2012,All genders,2415884.0


In [5]:
datafile.isnull().sum()

Country       0
Year          0
Gender        0
Enrolments    0
dtype: int64

In [6]:
datafile = datafile.sort_values(by = ["Country", "Year", "Gender"])

datafile.head(100)

,Country,Year,Gender,Enrolments
56,Afghanistan,1975,All genders,185723.0
55,Afghanistan,1975,Female,22281.0
54,Afghanistan,1976,All genders,204889.0
53,Afghanistan,1976,Female,26173.0
52,Afghanistan,1977,All genders,223379.0
...,...,...,...,...
90,Albania,1996,All genders,346945.0
89,Albania,1996,Female,168746.0
88,Albania,1997,All genders,350157.0
87,Albania,1997,Female,169918.0


In [7]:
datafile = datafile.reset_index(drop = True)
datafile.index = datafile.index +1 

datafile.head()

,Country,Year,Gender,Enrolments
1,Afghanistan,1975,All genders,185723.0
2,Afghanistan,1975,Female,22281.0
3,Afghanistan,1976,All genders,204889.0
4,Afghanistan,1976,Female,26173.0
5,Afghanistan,1977,All genders,223379.0


In [8]:
datafile.to_csv("../data_clean/secondary_clean.csv", index=True)

In [9]:
import os
os.path.exists("../data_clean")

True